# **Lab 4: Integration**
**Mirja Johnsson**

# **Abstract**

In lab 4 for *Methods in Scientific Computation* 3 methods for numerical integration were implemented. They are (Legendre-)Gauss 2-point, and 3-point quadrature, and Monte Carlo Integration.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license. 

In [ ]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**



[Hyperlink to DD2363 course website.](https://kth.instructure.com/courses/17068)

$
{\displaystyle \frac{\partial u}{\partial t}} + u\cdot \nabla u +\nabla p = f, \quad \nabla \cdot u=0$



In [12]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common 
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *
    
#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**
In this lab there are three mandatory assignments, each showcasing different techniques for numerical approximate integration. The first is a 2-point Gauss quadrature over a unit interval; the second is a 3-point edge midpoint quadrature over a reference triangle; the third is a Monte Carlo quadrature over a unit interval. 


# **Method**

To solve this week's assignments I have used methods described in chapters 11 and 12 in the course-book. For assignment 4.1 example 11.3 gives the quadrature points and the weights to be used when itegrating over the interval [-1,1]. We are asked to integrate over the interval [0,1] instead but new values can be plugged into the derived values to update them to the appropriate ones. I looked at this discussion thread to see how to update the x-values and weights. [länk](https://math.stackexchange.com/questions/1544918/gaussian-quadrature-with-a-to-0-1-reference-domain-instead-of-a-1-1-ref)


(I think it might also be possible to use the old values and integrate from [-1,1], and then divide the resulting value in 2. This could work on the basis that the Legendre polynomal ${P_2(x)}$ is symmetric about the y-axis? But I will not gamble with the assignment.)

When the integration was over the old interval the roots were 
${\displaystyle x_0 = \frac{1}{\sqrt 3}, x_1 = -\frac{1}{\sqrt 3}}$ but for the reason just described they were changed to ${\displaystyle \hat{x}_0 = \frac{x_0 + 1}{2}, \hat{x}_1 = \frac{x_1 + 1 }{2}}$. The weights were changed by halving them from 1 to 0.5.

In [16]:
#4.1
#Input: function f(x) 
#Output: quadrature approximation of integral int_0^1 f(x) dx
#Test: verify exact integration of cubic polynomials ax^3 + bx^2 + cx + d

def two_point_gauss(f):
  x_0 = (1.0/np.sqrt(3) +1) /2
  x_1 = (-1.0/np.sqrt(3)+1 )/2 
  return (fxdx(x_0) + fxdx(x_1)) / 2.0



From course book: 
(viii)  The 3-point edge midpoint quadrature rule for the reference triangle, with the weights w0=w1=w2=1/6, is exact for quadratic polynomials.

In [47]:
#4.2 
# 3-point edge midpoint quadrature over a reference triangle

#Input: function f(x) 
#Output: quadrature approximation of integral int_ref-triangle f(x) dx
#Test: verify exact integration of quadratic polynomials ax^2 + by^2 + cxy + dx + ey + f

def three_point_edge_midpoint_triangle(f):
  weights = 1/6
  return  (f(0.5, 0)+f(0,0.5)+f(0.5,0.5))*weights 


In [10]:
#4.3  
#Monte Carlo quadrature over a unit interval
#Output: MC quadrature approximation of integral int_0^1 f(x) dx
#Test: verify convergence rate 1/sqrt(n) with respect to number of quadrature points n.

def monte_carlo_integration(f, n):
    sum = 0
    for i in range(n):
        sum += f(np.random.uniform(0, 1))
    sum /= n    
    return sum 


# **Results**
I decided to test my integrating functions againts specific functions that I defined, instead of using any randomisation. This was because I felt more sure of having solved the implementations correctly when I knew exactly what result was expected to be returned. 

In [17]:
#Test 2-point Gauss quadrature

def fxdx(x):
  return 3*(x**2) + 6*x -1

def F(x):
  return x**3 + 3*(x**2) -x   

print("When we integrate f(x) dx from 0 to 1 using two-point Gaussian quadrature we expect the same value as we get when taking F(1) - F(0).\n Result of integration:",two_point_gauss(fxdx))
print("Result of F(1) - F(0): ",F(1.0) - F(0.0))



When we integrate f(x) dx from 0 to 1 using two-point Gaussian quadrature we expect the same value as we get when taking F(1) - F(0).
 Result of integration: 3.0
Result of F(1) - F(0):  3.0


In [48]:
#Test of 3-point edge midpoint quadrature over a reference triangle
#Generate 

def f_triangle(x, y):
  return 3*x**2 - 3*y**2 + x*y + 4*y - x + 1

print("I defined the function f_triangle that I integrated by hand and got the value 1.0417, also verified on wolframalpha.com. \n Compare this to the resulting value from integrating over the triangle: \n")  
print(round(three_point_edge_midpoint_triangle(f_triangle),6))

I defined the function f_triangle that I integrated by hand and got the value 1.0417, also verified on wolframalpha.com. 
 Compare this to the resulting value from integrating over the triangle: 

1.041667


This test uses the same function fxdx and primitive F(x) as the first test.

In [42]:
#Test of Monte Carlo Integration

#test
ns = [4,32, 128, 256, 512, 1024, 2048, 3000, 4000, 5000]
rate = np.zeros_like(ns, dtype = float)
sums = np.zeros_like(ns, dtype = float)
for n in range(len(ns)):
  value = round(monte_carlo_integration(fxdx, ns[n]), 6)
  sums[n] = value
  rate[n] = abs(value-3.0) /np.sqrt(ns[n]) 
print("The correct integrating value is as we already established 3.0 when integrating fxdx from 0 to 1 \n ")
print("\nThese are the values achieved by monte Carlo Integration, with number of samples ranging from 4 to 5000.\n", sums)  
print("\nThis is the distance of the value to the correct answer, divided by the square root of the number of samples n.\n", rate)
print("After several runs I can see the integration method starts to give reasonable results when we use 512 samples, but even then it still fluctuates too much. \n",
"1024 samples and up are more stable.")

The correct integrating value is as we already established 3.0 when integrating fxdx from 0 to 1 
 

These are the values achieved by monte Carlo Integration, with number of samples ranging from 4 to 5000.
 [2.519468 2.625866 2.741486 2.995431 3.060373 2.941628 3.031342 3.028981
 3.072062 3.028309]

This is the distance of the value to the correct answer, divided by the square root of the number of samples n.
 [0.240266   0.06613817 0.02284963 0.00028556 0.00266813 0.00182412
 0.00069257 0.00052912 0.0011394  0.00040035]
After several runs I can see the integration method starts to give reasonable results when we use 512 samples, but even then it still fluctuates too much. 
 1024 samples and up are more stable.


# **Discussion**
This was a tricky assignment, but once I started to understand more about why we could do what we were supposed to do it became very interesting. I read that the "naive" method of sampling randomly-uniformly from the domain can be improved upon. By looking at the function and drawing more samples from the area where the probability density function is concentrated you will converge faster towards the correct integral. This is for example made us of when rendering scenes with Monte Carlo ray tracing.